In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Giả lập một văn bản dài (Ví dụ: Tài liệu hướng dẫn)
long_text = """
Docker là một nền tảng mở để phát triển, vận chuyển và chạy các ứng dụng. 
Docker cho phép bạn tách ứng dụng khỏi cơ sở hạ tầng để bạn có thể phân phối phần mềm nhanh chóng. 
Với Docker, bạn có thể quản lý cơ sở hạ tầng của mình giống như cách bạn quản lý các ứng dụng.
Qdrant là một cơ sở dữ liệu vector. Nó được tối ưu hóa để lưu trữ và truy vấn các vector embedding.
"""

# 2. Khởi tạo bộ chia (Splitter)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,      # Kích thước mỗi đoạn (ký tự)
    chunk_overlap=20,    # Độ chồng lấn (để giữ ngữ cảnh giữa các đoạn cắt)
    length_function=len,
)

# 3. Thực hiện cắt
docs = text_splitter.create_documents([long_text])

print(f"Đã cắt thành {len(docs)} đoạn.")
print(f"Ví dụ đoạn 1: {docs[1].page_content}")

Đã cắt thành 4 đoạn.
Ví dụ đoạn 1: Docker cho phép bạn tách ứng dụng khỏi cơ sở hạ tầng để bạn có thể phân phối phần mềm nhanh chóng.


In [6]:
from sentence_transformers import SentenceTransformer

# 1. Tải model (all-MiniLM-L6-v2 là model nhỏ, nhanh, phổ biến cho demo)
model = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Lấy nội dung text từ các đoạn đã cắt ở bước 1
texts = [d.page_content for d in docs]

# 3. Biến đổi thành vector
embeddings = model.encode(texts)

# Kiểm tra kích thước vector (Thường là 384 chiều với model này)
vector_size = len(embeddings[1]) 
print(f"Kích thước vector: {vector_size}")

Kích thước vector: 384


In [7]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
import uuid

# 1. Khởi tạo Qdrant (Dùng ":memory:" để chạy trên RAM, không cần cài server)
client = QdrantClient(":memory:")

# 2. Tạo Collection (giống như tạo Bảng trong SQL)
collection_name = "my_knowledge_base"
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
)

# 3. Đưa dữ liệu vào Qdrant
points = []
for idx, (text, vector) in enumerate(zip(texts, embeddings)):
    points.append(
        PointStruct(
            id=idx,  # ID có thể là số hoặc UUID
            vector=vector.tolist(),
            payload={"text": text}  # Lưu lại text gốc để tí nữa lấy ra đọc
        )
    )

client.upsert(
    collection_name=collection_name,
    points=points
)
print("Đã lưu dữ liệu vào Qdrant thành công!")

Đã lưu dữ liệu vào Qdrant thành công!


C:\Users\HP\AppData\Local\Temp\ipykernel_62396\3989165896.py:10: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


In [8]:
# 1. Câu hỏi của người dùng
query = "Docker dùng để làm gì?"

# 2. Mã hóa câu hỏi thành vector (dùng chung model với lúc nạp dữ liệu)
query_vector = model.encode(query).tolist()

# 3. Tìm kiếm trong Qdrant
search_result = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=2  # Lấy 2 kết quả tốt nhất
)

# 4. In kết quả
print(f"\nCâu hỏi: {query}")
print("-" * 30)
for result in search_result:
    print(f"Độ tương đồng: {result.score:.4f}")
    print(f"Nội dung tìm thấy: {result.payload['text']}")
    print("-" * 30)


Câu hỏi: Docker dùng để làm gì?
------------------------------
Độ tương đồng: 0.6241
Nội dung tìm thấy: Docker là một nền tảng mở để phát triển, vận chuyển và chạy các ứng dụng.
------------------------------
Độ tương đồng: 0.5813
Nội dung tìm thấy: Docker cho phép bạn tách ứng dụng khỏi cơ sở hạ tầng để bạn có thể phân phối phần mềm nhanh chóng.
------------------------------


In [12]:
print(f"Món ăn: 'name_vn' ('name_en'). "
f"Thuộc loại: cat_name. "
f"Giá: 'price' VND. "
f"Ghi chú: 'note'.")

Món ăn: 'name_vn' ('name_en'). Thuộc loại: cat_name. Giá: 'price' VND. Ghi chú: 'note'.


In [14]:
import json
import os
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance

# --- PHẦN 1: GIẢ LẬP MÔI TRƯỜNG & DỮ LIỆU ---
# 1. Tạo Config giả
class Config:
    EMBEDDING_MODEL = 'all-MiniLM-L6-v2' # Model nhỏ chạy cho nhanh
    COLLECTION_NAME = 'demo_collection'
    DATA_DIR = '.' # Lưu file tạm ngay thư mục hiện tại

# 2. Tạo file menu.json giả
sample_menu = [
    {
        "category": "Món Chính",
        "items": [
            {"id": "mc_01", "name_vn": "Phở Bò", "name_en": "Beef Noodle Soup", "price": 50000, "note": "Đặc biệt thơm ngon"},
            {"id": "mc_02", "name_vn": "Bún Chả", "name_en": "Grilled Pork Noodles", "price": 60000, "note": "Nước chấm gia truyền"}
        ]
    }
]
with open('menu.json', 'w', encoding='utf-8') as f:
    json.dump(sample_menu, f, ensure_ascii=False, indent=2)

# 3. Tạo file restaurant_info.txt giả
sample_info = """
Nhà hàng mở cửa từ 8:00 sáng đến 10:00 tối.
Chúng tôi miễn phí gửi xe cho khách hàng.
"""
with open('restaurant_info.txt', 'w', encoding='utf-8') as f:
    f.write(sample_info)

print("✅ Đã tạo xong dữ liệu mẫu (menu.json, restaurant_info.txt)")
print("-" * 50)

# --- PHẦN 2: LOGIC CỦA BẠN (CÓ THÊM PRINT ĐỂ SOI DỮ LIỆU) ---

class DataIngestor:
    def __init__(self, qdrant_client):
        self.client = qdrant_client
        print(f"⏳ Đang tải model {Config.EMBEDDING_MODEL}...")
        self.encoder = SentenceTransformer(Config.EMBEDDING_MODEL)
        print("✅ Model đã sẵn sàng!\n")
        
    def load_menu(self, path):
        try:
            with open(path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            print(f"👀 [Giai đoạn 1 - Đọc File] Đã đọc menu.json: Tìm thấy {len(data)} danh mục.")
            
            docs = []
            for category in data:
                cat_name = category['category']
                for item in category['items']:
                    # Tạo nội dung ngữ nghĩa
                    content = (
                        f"Món ăn: {item['name_vn']} ({item['name_en']}). "
                        f"Thuộc loại: {cat_name}. "
                        f"Giá: {item['price']} VND. "
                        f"Ghi chú: {item['note']}."
                    )
                    
                    # IN RA ĐỂ KIỂM TRA
                    print(f"   👉 [Giai đoạn 2 - Biến đổi] JSON -> Text: \"{content}\"")
                    
                    docs.append({"text": content, "source": "menu", "id": item['id']})
            return docs
        except FileNotFoundError:
            print("Warning: Menu file not found.")
            return []

    def load_info(self, path):
        try:
            with open(path, 'r', encoding='utf-8') as f:
                content = f.read()
            
            # Chia nhỏ
            chunks = content.split('\n')
            docs = []
            print(f"👀 [Giai đoạn 1 - Đọc File] Đã đọc info.txt.")
            
            for i, chunk in enumerate(chunks):
                if chunk.strip():
                    # IN RA ĐỂ KIỂM TRA
                    print(f"   👉 [Giai đoạn 2 - Biến đổi] Raw Text -> Chunk: \"{chunk.strip()}\"")
                    docs.append({"text": chunk.strip(), "source": "info", "id": f"info_{i}"})
            return docs
        except FileNotFoundError:
            print("Warning: Info file not found.")
            return []

    def ingest(self):
        print("--- Bắt đầu nạp dữ liệu ---")
        menu_docs = self.load_menu(os.path.join(Config.DATA_DIR, 'menu.json'))
        info_docs = self.load_info(os.path.join(Config.DATA_DIR, 'restaurant_info.txt'))
        
        all_docs = menu_docs + info_docs
        
        if not all_docs:
            print("Không có dữ liệu.")
            return

        # Tạo collection (RAM)
        self.client.recreate_collection(
            collection_name=Config.COLLECTION_NAME,
            vectors_config=VectorParams(size=384, distance=Distance.COSINE),
        )

        print(f"\n⚡ [Giai đoạn 3 - Vector hóa] Đang biến đổi {len(all_docs)} đoạn văn thành số...")
        embeddings = self.encoder.encode([d['text'] for d in all_docs])
        
        # In thử vector đầu tiên xem mặt mũi nó ra sao
        print(f"   👉 Vector mẫu (3 số đầu/384): {embeddings[0][:3]} ...")

        points = []
        for i, doc in enumerate(all_docs):
            points.append(PointStruct(
                id=i, # Lưu ý: Code gốc của bạn dùng i làm ID, cẩn thận trùng nếu chạy nhiều lần
                vector=embeddings[i].tolist(),
                payload=doc
            ))
            
        print(f"\n📦 [Giai đoạn 4 - Đóng gói] Chuẩn bị đẩy {len(points)} Points vào Qdrant.")
        
        self.client.upsert(
            collection_name=Config.COLLECTION_NAME,
            points=points
        )
        print(f"✅ [HOÀN TẤT] Đã nạp thành công vào Qdrant Memory!")

# --- PHẦN 3: CHẠY THỬ ---
if __name__ == "__main__":
    # Dùng Qdrant chạy trên RAM (không cần cài Docker để test)
    client = QdrantClient(":memory:")
    
    ingestor = DataIngestor(client)
    ingestor.ingest()
    
    # Test thử tìm kiếm để chứng minh dữ liệu đã vào
    print("\n--- 🔎 TEST TÌM KIẾM ---")
    query = "Nhà hàng bán món gì 50000?"
    print(f"Câu hỏi: {query}")
    
    model = SentenceTransformer(Config.EMBEDDING_MODEL)
    hits = client.search(
        collection_name=Config.COLLECTION_NAME,
        query_vector=model.encode(query).tolist(),
        limit=1
    )
    
    for hit in hits:
        print(f"Kết quả tìm thấy: {hit.payload['text']}")

✅ Đã tạo xong dữ liệu mẫu (menu.json, restaurant_info.txt)
--------------------------------------------------
⏳ Đang tải model all-MiniLM-L6-v2...
✅ Model đã sẵn sàng!

--- Bắt đầu nạp dữ liệu ---
👀 [Giai đoạn 1 - Đọc File] Đã đọc menu.json: Tìm thấy 1 danh mục.
   👉 [Giai đoạn 2 - Biến đổi] JSON -> Text: "Món ăn: Phở Bò (Beef Noodle Soup). Thuộc loại: Món Chính. Giá: 50000 VND. Ghi chú: Đặc biệt thơm ngon."
   👉 [Giai đoạn 2 - Biến đổi] JSON -> Text: "Món ăn: Bún Chả (Grilled Pork Noodles). Thuộc loại: Món Chính. Giá: 60000 VND. Ghi chú: Nước chấm gia truyền."
👀 [Giai đoạn 1 - Đọc File] Đã đọc info.txt.
   👉 [Giai đoạn 2 - Biến đổi] Raw Text -> Chunk: "Nhà hàng mở cửa từ 8:00 sáng đến 10:00 tối."
   👉 [Giai đoạn 2 - Biến đổi] Raw Text -> Chunk: "Chúng tôi miễn phí gửi xe cho khách hàng."

⚡ [Giai đoạn 3 - Vector hóa] Đang biến đổi 4 đoạn văn thành số...
   👉 Vector mẫu (3 số đầu/384): [-0.13363892  0.06800253 -0.03199044] ...

📦 [Giai đoạn 4 - Đóng gói] Chuẩn bị đẩy 4 Points vào Qdran

C:\Users\HP\AppData\Local\Temp\ipykernel_62396\1969296676.py:107: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  self.client.recreate_collection(


Kết quả tìm thấy: Nhà hàng mở cửa từ 8:00 sáng đến 10:00 tối.
